In [1]:
%run stdPackages.ipynb

Data:

In [2]:
dates_log = pd.Index([1950, 1980, 2010, 2040, 2070, 2100], name = 't')
ν_log = np.array([1.497025777, 1.365562914, 1.20756547, 1.110843373, 1.072547602, 1.0725])
T_log = len(ν_log) # number of years in data
T = T_log+5
dates = dates_log.union(pd.Index([dates_log[-1]+30*i for i in range(1,T-T_log+1)]))
ν = np.hstack([ν_log, np.full(T-T_log, ν_log[-1])])
A = np.ones(T) # normalize exog. productivity at 1 
α = .43 # capital income share
ξ = .35 # frisch elasticity
γ = np.full((4, ), 0.25) # four equally large shares
ni = len(γ) # number of types of working households in this case
hours = np.array([38.6, 41.8, 43.4, 46.8]) # weekly hours, working households
income= np.array([618.6, 945.5, 1278.6, 2341.6]) # income levels, working households
zx = hours/hours.mean() # normalized data
zη = income/income.mean() # normalized data
γu =.32 # share of u households
χ1 = 0.3089 # relative income of u-types
χ2 = 0.95 * χ1 # relative income of u-types when old - assumed slightly lower
τ0 = 0.142 # target level of pension tax
s0 = 0.184 # target savings rate
t0date = 2010
t0 = dates.get_loc(t0date) # index for year of calibration 
RR = 0.678/0.803 # replacement rate
ω = 5
ρ = .5
ωu = .3
ωη = 1.65

Find eigenvectors:

In [3]:
valx, vecx = scipy.sparse.linalg.eigs(zx.reshape(ni,1) * γ.reshape(1,ni), k=1)
valη, vecη = scipy.sparse.linalg.eigs(zη.reshape(ni,1) * γ.reshape(1,ni), k=1)
yx, yη = abs(np.real(vecx)).reshape(ni), abs(np.real(vecη)).reshape(ni) # this assumes that all coordinates in eigenvector are either positive or negative; this should be the case

Calibrate parameters:

In [4]:
η = yη/(yx*sum(γ*yη))
X = η/yx**(1/ξ)

Now given parameters that define household heterogeneity, we define $\theta$ from the relative replacement rates:

In [5]:
Q1 = η[0]**(1+ξ)/(X[0]**ξ)
Q2 = η[1]**(1+ξ)/(X[1]**ξ)
θ = (RR/Q1-1/Q2)/(1-RR+RR/Q1-1/Q2)

Set uniform $\beta$ for all types:

In [6]:
β = np.full(ni, fill_value = 1)
βu= min(β) # set impatience equal to lowest producticity household. 

Given our guess on $\beta$, we can define $\epsilon$ from the built-in function for the Argentina case: 

In [7]:
epsilon = CRRA_inf.argentinaCalEps(θ, β[0])

## 2. Initialize and calibrate model

Grid settings:

In [8]:
ngrid = 25 # number of gridpoints in the savings grid
_min = 1e-3 # use as "small" number instead of 0
exp = 1 # nonlinearity in grid, exp>1 increases number of gridpoints in the lower end of the grid

Initialize CRRA and log models:

In [9]:
kwargs = {'α': α, 'A': A, 'ν': ν, 'η': η, 'γ': γ, 'X': X, 'β': β, 'βu': βu, 'ξ': ξ, 'epsilon': epsilon, 'θ': θ, 'γu': γu, 'χ1': χ1, 'χ2': χ2, 'ρ': ρ, 'ω': ω, 'ωu': ωu,'ωη': ωη}
model = CRRA_inf_v2.infHorizon(ni=ni, T= T, ngrid = ngrid, **kwargs)
sGrid = CRRA_inf_v2.polGrid(_min, model.solve_ss(0,0,0,ν[-1])['s'], model.ngrid, exp)

In [10]:
self = model
from CRRA_inf_v2 import *

In [11]:
τ = np.full(self.ngrid, 0.2)

In [12]:
test = self.solve_ssVec_v2(τ, θ, epsilon, ν[-1])

Calibration procedure:

0. Calibrate to simple PEE first.
1. Identifying *some* PEE with ESC (i.e. for some parameter values). Retrieve policy functions as well as solutions (to get initial value for steady state policies).  
2. Calibrate model to target policy and savings rates (the simplest ones) with initial guess from above.
3. On a grid of $\omega_u, \omega_{\eta}$ as a globalization strategy. Keep updating the initial guess. 
4. With a fixed initial guess for terminal state policy (from the solution with parameter value close to the true solution), calibrate to all targets now.

First, define PEE and ESC functions that return not only the solution paths, but also policy functions on the grid of $t$.

In [ ]:
# model.argentinaCal_simple_PEE(τ0, s0, t0, sGrid) # calibrate without endogenous political choice

Next, get an initial solution:

In [ ]:
# sol, policy = model.solve_ESCBound(sGrid, returnPols=True)